In [3]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path

In [51]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_nodes(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, node_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(node_list), len(node_list)))
  for i in node_list:
    voisins_array = np.delete(node_list, np.where(node_list == i))
    for j in voisins_array:
      distance, path = get_distance_between_nodes(converted_graph, i, j)
      converted_graph_i = np.where(node_list == i)[0][0]
      converted_graph_j = np.where(node_list == j)[0][0]
      complete_graph[converted_graph_i][converted_graph_j] = distance
      all_paths[(converted_graph_i, converted_graph_j)] = path
  return complete_graph, all_paths

def get_random_vertex(size, nb_vertex):
  random_vertex = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertex.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertex)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return(shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, nb_vertex_chosen, density):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  random_vertex = get_random_vertex(nb_vertex_matrix, nb_vertex_chosen)
  complete_graph, all_paths = get_complete_graph(graph, random_vertex)
  interval = generate_interval()
  return graph, complete_graph, all_paths, random_vertex, interval

In [52]:
graph, complete_graph, all_paths, random_vertex, interval =\
generate_instance(
  nb_vertex_matrix = 1000, 
  min_weight = 10, 
  max_weight = 60, 
  nb_vertex_chosen = 100, 
  density = 0.3
)

graph ok
[884 318 499 678 235   5 411 451 778 754 494 345 342 383  23 219 336 644
 833 979 819 883 391  53 813 599 261 211 466 761 948 919  69 329 735 286
 821 279 934 815 463 363 267 858 548 803 755 333 343 656 891  90 862 205
 697 280 214 120 723 709 106 937 361 609 405  24 645 498 317 526  92 760
 123 189 311 725  50 631 816 900 955 911 820 552  94 354 150 814 320 358
 998 827 541  19 839 359 199 305 228 392]
random vertex ok
884
318
499
678
235
5
411
451
778
754
494
345
342
383
23
219
336
644
833
979
819
883
391
53
813
599
261
211
466
761
948
919
69
329
735
286
821
279
934
815
463
363
267
858
548
803
755
333
343
656
891
90
862
205
697
280
214
120
723
709
106
937
361
609
405
24
645
498
317
526
92
760
123
189
311
725
50
631
816
900
955
911
820
552
94
354
150
814
320
358
998
827
541
19
839
359
199
305
228
392
complete graph ok
interval ok


In [47]:
print("graph")
print(graph)
print("complete_graph")
print(complete_graph)
print("all_paths")
print(all_paths)
print("random_vertex")
print(random_vertex)
print("interval")
print(interval)

graph
[[inf inf 35. 39. inf inf]
 [inf inf 22. 27. 36. 50.]
 [35. 22. inf inf inf 40.]
 [39. 27. inf inf 47. inf]
 [inf 36. inf 47. inf inf]
 [inf 50. 40. inf inf inf]]
complete_graph
[[ 0. 47. 77. 27.]
 [47.  0. 86. 36.]
 [77. 86.  0. 50.]
 [27. 36. 50.  0.]]
all_paths
{(0, 1): [3, 4], (0, 2): [3, 1, 5], (0, 3): [3, 1], (1, 0): [4, 3], (1, 2): [4, 1, 5], (1, 3): [4, 1], (2, 0): [5, 1, 3], (2, 1): [5, 1, 4], (2, 3): [5, 1], (3, 0): [1, 3], (3, 1): [1, 4], (3, 2): [1, 5]}
random_vertex
[3 4 5 1]
interval
(480, 540)


In [13]:
# from pulp import *
# import numpy as np
# X=[]
# # variables du problème
# for i in range(len(complete_graph)):
#     for j in range(len(complete_graph)):
#         X.append(LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger))



# # probleme
# prob = LpProblem("plus court chemin", LpMinimize)

# # objectif
# prob += lpSum([complete_graph[i][j] * X[i*len(complete_graph) + j] for i in range(len(complete_graph)) for j in range(len(complete_graph))])


# # contraintes
# prob += sum([X[i,j] for i in range(len(complete_graph)) for j in range(len(complete_graph))]) == 1


# prob.solve()
# print(LpStatus[prob.status])
# print("Min=", value(prob.objective))

# # variables resultat
# for v in prob.variables():
#     print("%s=%.2f"%(v.name,v.varValue), end=', ')
